#  Work with Sequences

Up until now, we’ve focused on models whose inputs consisted of a single feature vector $\mathbf{x} \in \mathbb{R}^d$.

The main change of perspective when developing models capable of processing sequences is that we now focus on inputs that consist of an ordered list of feature vectors  $\mathbf{x}_1, \dots, \mathbf{x}_T$, where each feature vector $\mathbf{x}_t$ indexed by a time step $t \in \mathbb{Z}^+$ lies in $\mathbb{R}^d$.

Some datasets consist of
1. A single massive sequence. Consider, for example, the extremely long streams of sensor readings that might be available to climate scientists. In such cases, we might create training datasets by randomly sampling subsequences of some predetermined length.
2. More often, our data arrives as a collection of sequences. Consider the following examples:
    * A collection of documents, each represented as its own sequence of words, and each having its own length $T_i$.
    * Sequence representation of patient stays in the hospital, where each stay consists of a number of events and the sequence length depends roughly on the length of the stay.

While we still assume that entire sequences (e.g., entire documents or patient trajectories) are sampled independently, we cannot assume that the data arriving at each time step are independent of each other. For example, what words are likely to appear later in a document depends heavily on what words occurred earlier in the document. What medicine a patient is likely to receive on the 10th day of a hospital visit depends heavily on what transpired in the previous nine days.

This should come as no surprise. If we didn’t believe that the elements in a sequence were related, we wouldn’t have bothered to model them as a sequence in the first place. Consider the usefulness of the auto-fill features that are popular on search tools and modern email clients. They are useful precisely because it is often possible to predict (imperfectly, but better than random guessing) what likely continuations of a sequence might be, given some initial prefix. For most sequence models, we don’t require independence, or even stationarity, of our sequences. Instead, we require only that the sequences themselves are sampled from some fixed underlying distribution over entire sequences.

This flexible approach, allows for such phenomena as:
1. Documents looking significantly different at the beginning than at the end
2. Patient status evolving either towards recovery or towards death over the course of a hospital stay
3. Customer taste evolving in predictable ways over course of continued interaction with a recommender system.


We sometimes wish to predict a fixed target $y$ given sequentially structured input (e.g., sentiment classification based on a movie review).
At other times, we wish to predict a sequentially structured target ($y_1, \ldots, y_T$) given a fixed input (e.g., image captioning).
Still other times, our goal is to predict sequentially structured targets based on sequentially structured inputs (e.g., machine translation or video captioning).

Such sequence-to-sequence tasks take two forms:
1. *aligned*: where the input at each time step aligns with a corresponding target (e.g., part of speech tagging).
2. *unaligned*: where the input and target do not necessarily exhibit a step-for-step correspondence (e.g., machine translation).


Before we worry about handling targets of any kind, we can tackle the most straightforward problem: unsupervised density modeling (also called sequence modeling). Here, given a collection of sequences, our goal is to estimate the probability mass function that tells us how likely we are to see any given sequence, i.e., $p(\mathbf{x}_1, \ldots, \mathbf{x}_T)$.



# 1. Autoregressive Models

Before introducing specialized neural networks designed to handle sequentially structured data, let’s take a look at some actual sequence data and build up some basic intuitions and statistical tools.

<img src='img_1.png'>

In particular, we will focus on stock price data from the FTSE 100 index.  At each *time step* $t \in \mathbb{Z}^+$, we observe
the price of the index at that time, denoted by $x_t$.

Now suppose that a trader would like to make short term trades, strategically getting into or out of the index, depending on whether they believe that it will rise or decline in the subsequent time step. The trader is thus interested in knowing the probability distribution:
$$P(x_t \mid x_{t-1}, \ldots, x_1)$$

over prices that the index might take in the subsequent time step. While estimating the entire distribution over a continuous-valued random variable can be difficult, the trader would be happy to focus on a few key statistics of the distribution, particularly the expected value and the variance. One simple strategy for estimating the conditional expectation:
$$\mathbb{E}[(x_t \mid x_{t-1}, \ldots, x_1)],$$
would be to apply a linear regression model

Such models that regress the value of a signal on the previous values of that same signal are naturally called autoregressive models.

There is just one major problem: the number of inputs, $x_{t-1}, \ldots, x_1$ depending on *t*. Namely, the number of inputs increases with the amount of data that we encounter. Thus if we want to treat our historical data as a training set, we are left with the problem that each example has a different number of features.


A few strategies recur frequently:
1. We might believe that although long sequences $x_{t-1}, \ldots, x_1$ are available, it may not be necessary to look back so far in the history when predicting the near future.
    * In this case we might content ourselves to condition on some window of length $\tau$ and only use $x_{t-1}, \ldots, x_{t-\tau}$ observations.
    * The immediate benefit is that now the number of arguments is always the same, at least for $t > \tau$.
    * This allows us to train any linear model or deep network that requires fixed-length vectors as inputs.

2. we might develop models that maintain some summary $h_t$ of the past observations and at the same time update $h_t$ in addition to the prediction $\hat{x_t}$
<img src='img_2.png'>

This leads to models that estimate $x_t$ with $\hat{x}_t = P(x_t \mid h_{t})$ and moreover updates of the form $h_t = g(h_{t-1}, x_{t-1})$. Since $h_t$ is never observed, these models are also called *latent autoregressive models*.

# 2 Sequence Model
Sometimes, especially when working with language, we wish to estimate the joint probability of an entire sequence.
This is a common task when working with sequences composed of discrete tokens, such as words. Generally, these estimated functions are called sequence models and for natural language data, they are called language models,  even when dealing with non-language data

Language models prove useful for all sorts of reasons. Sometimes we want to evaluate the likelihood of sentences.

1. For example, we might wish to compare the naturalness - Tính tự nhiên - of two candidate outputs generated by a machine translation system or by a speech recognition system. But language modeling gives us not only the capacity to evaluate likelihood, but the ability to sample sequences, and even to optimize for the most likely sequences.

While language modeling might not look, at first glance, like an autoregressive problem, we can reduce language modeling to autoregressive prediction by decomposing the joint density  of a sequence $p(x_t \mid x_1, \ldots, x_T)$ into the product of conditional densities
in a left-to-right fashion by applying the chain rule of probability:

$$P(x_1, \ldots, x_T) = P(x_1) \prod_{t=2}^T P(x_t \mid x_{t-1}, \ldots, x_1).$$

Note that if we are working with **discrete signals** like words, then the autoregressive model must be a **probabilistic classifier**, **outputting** a full **probability distribution** over the vocabulary for what word will come next, given the leftwards context.

## 2.1 Markov Models
